# Python advanced class

## Module 9, Test and Software Quality

* Pytest
* Logging
* mypy
* Ruff format
* Ruff check
* precommit

# Pytest

## Pytest, the simple unit test tool

* `pytest` is the easier replacement of the stdlib module `unittest`
* `pytest` relies on naming conventions and the `assert` keyword
* With `pytest` there is no excuse for not writing tests!

## mymodule.py and test_module.py

* Given a module `mymodule.py` with a function `f()`:

```python
# mymodule.py
def f():
    return "This is f"
```
* We now create `test_mymodule.py` with a function `test_f()`:

```python
# test_mymodule.py
import mymodule
def test_f():
    assert mymodule.f() == "This is f"
```



## Running pytest

* Running `pytest` can be runned without arguments, or
* With a directory like `tests/`
* With a specific file like `tests/test_mymodule.py`
* With a specific test function: `tests/test_mymodule.py::test_f`

```shell
shell> pytest tests/test_mymodule.py::test_f
tests/test_mymodule.py .               [100%]
============= 1 passed in 0.01s =============
```


## Configuring pytest in pyproject.toml

* To configure pytest, you can add the following lines to your `pyproject.toml` file:

```toml
[tool.pytest.ini_options]
testpaths = ["tests"]
python_files = ["test_*.py", "*_test.py"]
addopts = "-v --disable-warnings"
```

## pytest.mark

* We can write tests for code that isn't written yet!
* If the function is not defined we can mark the test `@pytest.mark.skip`
* If the function is not working we can mark the test `@pytest.mark.xfail`
* We don't want to get used to see failing tests we are not working on now!

```python
@pytest.mark.skip(reason="g is not implemented yet")
def test_g():
    assert mymodule.g() == "This is g"

@pytest.mark.xfail(reason="f is implemented incorrectly")
def test_f_fail():
    assert mymodule.f() == "This is the function f"
```

## Parametrizing test sets

* Pytest provides a powerful way to create reusable test setups.
* Fixtures can help test a function with a set of test values.
* First argument is a string of names for the parameters
* The second is a list of tuples with adhering values

```python
@pytest.mark.parametrize(
    "a, b, expected",
    [
        (3, 4, 5.0),
        (5, 12, 13.0),
        (8, 15, 17.0),
    ],
)
def test_pyt(a, b, expected):
    assert mymodule.pyt(a, b) == expected
```


## Mocking

* Mocking makes it possible to test functions that call slow or changing libraries
* A mock is a simulation of the library function used
* There are a number of ways to apply mocking, but most used are `unittest.mock` and `pytest.monkeypatch`

```python
def get_yr_weather_temperature(lat: float, lon: float) -> float:
    """Fetch weather data for a given location from the Yr API."""
    base_url = "https://api.met.no/weatherapi/locationforecast/2.0/compact"
    url = f"{base_url}?lat={lat}&lon={lon}"
    headers = {"Accept": "application/json", "User-Agent": "mymodule-example/1.0"}
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    temperature = response.json()["properties"]["timeseries"][0]["data"]["instant"][
        "details"
    ]["air_temperature"]
    return temperature
```

```python
def test_get_yr_weather_temperature_mock(monkeypatch):
    class MockResponse:
        @staticmethod
        def json():
            return {"properties": {"timeseries": [
                {"data": {"instant": {"details": {"air_temperature": 20.5}}}}
            ]}}
        @staticmethod
        def raise_for_status():
            pass
    def mock_get(*args, **kwargs):
        return MockResponse()
    monkeypatch.setattr(mymodule.requests, "get", mock_get)
    temperature = mymodule.get_yr_weather_temperature(55.68888, 12.5)
    assert temperature == 20.5
```